In [2]:
import geemap 
import ee 
ee.Initialize()
#ee.Authenticate()

In [3]:
Map = geemap.Map()
#roi01 = geemap.shp_to_ee(r"F:\随机森林\土壤湿度反演\region\area_gcs.shp").geometry()
roi = ee.Geometry.Rectangle([113, 26, 119, 30], None, False)
Map.addLayer(roi, {}, 'Boundary')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
startDate = ee.Date.fromYMD(2023,1,1)
endDate = ee.Date.fromYMD(2023,12,31)
#Sentinel-2 去云
def maskCloudAndShadowsSR(image):
  cloudProb = image.select('MSK_CLDPRB')
  snowProb = image.select('MSK_SNWPRB')
  cloud = cloudProb.lt(5)
  snow = snowProb.lt(5)
  scl = image.select('SCL')
  shadow = scl.eq(3); # 3 = cloud shadow
  cirrus = scl.eq(10); # 10 = cirrus
  # Cloud probability less than 5% or cloud shadow classification
  mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
  return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])
#function: EVI for S2
def funEVIS2(image,B1,B2,B3):
  VI = image.expression('2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
      {
        'blue': image.select(B1),
        'red':  image.select(B2),
        'nir':  image.select(B3),
      }).rename('EVI')
  return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
def addVIs(img):
  nir = img.select('B8')
  red = img.select('B4')
  blue = img.select('B2')
  evi = funEVIS2(img,'B2','B4','B8')
  ndvi = img.normalizedDifference(['B8','B4']).rename('NDVI')
  lswi = img.normalizedDifference(['B8','B11']).rename('LSWI')
  mndwi = img.normalizedDifference(['B3','B11']).rename('mNDWI')
  snow = mndwi.gt(0.4).And(nir.gt(0.11))
  img =img.addBands(evi).addBands(ndvi).addBands(lswi).addBands(mndwi).updateMask(snow.eq(0))
  # return img.select(['LSWI','EVI','NDVI','mNDWI'])
  return img.select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','LSWI','EVI','NDVI','mNDWI'])
#纹理信息提取
def texture(image):
  text = image.divide(10000).toInt32().glcmTexture() #归一化
  texture_feature = text.select(['NDVI_contrast','NDVI_corr','NDVI_idm','NDVI_ent'])
  return image.addBands(texture_feature)


In [1]:
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR')
images = s2Sr.filterDate(startDate, endDate) \
                     .filterBounds(roi) \
                     .map(maskCloudAndShadowsSR) \
                     .map(addVIs) \
                     .map(texture)
composite = images.reduce(ee.Reducer.percentile([10,50,90]))
#geemap.download_ee_image(composite.clip(roi), r"F:\随机森林\土壤湿度反演\img\composite.tif", region=roi, crs='epsg:4326', scale=30)
type(composite)

NameError: name 'ee' is not defined

In [ ]:
startDate = ee.Date.fromYMD(2023,1,1)
endDate = ee.Date.fromYMD(2023,12,31)
###############Train the classifier using data 2023#############
empty = ee.Image().byte()
outline = empty.paint(**{
  'featureCollection': roi,
  'color': 'yellow',
  'width': 3
})
Map.addLayer(outline,{},"outline",False)
###Data preparation###
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR')
# filter data and location
images = s2Sr.filterDate(startDate, endDate) \
                     .filterBounds(roi) \
                     .map(maskCloudAndShadowsSR) \
                     .map(addVIs) \
                     .map(texture)\
                   
#print('Sentinel-2 Level 2')
# print(images.size())
# print(images.first().bandNames())

#Step 1: Bands management
composite = images.reduce(ee.Reducer.percentile([10,50,90]))
# print("composite",composite.bandNames())
#Input variables
texture_bands = composite.bandNames().slice(39,51)# 24 or 51
texture_image = composite.select(texture_bands)
spectral_bands = composite.bandNames().slice(0,39)#12 or 39
spectral_image = composite.select(spectral_bands)
feature_bands = composite.bandNames()
feature_image = spectral_image.addBands(texture_image)
# print(spectral_bands)
# print(texture_bands)
# print(feature_bands)

#####Step2: training data########
# all_training = Mangrove_sample.merge(Saltmarsh_sample)
# # validation = Spartina_vad.merge(Nonspartina_vad)
# print('all_training',all_training.size())
training = Mangrove_sample.merge(Inland_sample).filterBounds(roi)
print('training',training.size())

training_samples = feature_image.sampleRegions(**{
  'collection': training,
  'properties': ['Class'],
  'scale': 10,
  'tileScale': 16
})

#print(training_samples.size(),testing_samples.size())
trainingData = training_samples.randomColumn('random');#
sample_training = trainingData.filter(ee.Filter.lte("random", 0.7))
sample_validate = trainingData.filter(ee.Filter.gt("random", 0.7))

classifier = ee.Classifier.smileRandomForest(50).train(**{
  'features': trainingData,
  'classProperty': 'Class',
  'inputProperties': feature_bands
  })

# #对sentinel进行分类
class_img = feature_image.clip(roi).classify(classifier)
# m.addLayer(class_img.clip(southLine).updateMask(Guangxi_veget), {min:1,max:2, palette: ['yellow', 'green']},'original_veget',False)

# #Step 4: Assess
# explain = classifier.explain()
# print('Explain',explain)

#Save the classifier
trees = ee.List(ee.Dictionary(classifier.explain()).get('trees'))
dummy = ee.Feature(roi.geometry(),None)

def func_nfp(x)return dummy.set('tree',x)});:
colTree = ee.FeatureCollection(trees.map(function(x){return dummy.set('tree',x)}
colTree = ee.FeatureCollection(trees.map(func_nfp
))
#print(colTree)
Export.table.toAsset(colTree,'PapuaNewGuinea_tidal_vegetation_save_classifier2022','projects/geemap-2022/assets/PV_classifier')